In [1]:
!nvidia-smi

Mon Jan 18 22:27:08 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [8]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras import layers, callbacks, utils
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
import glob
import os

In [12]:
!cp -r /content/drive/MyDrive/cleaned_data/ /content/
!cp -r /content/drive/MyDrive/noisy_data/ /content/

In [14]:
cleaned = glob.glob("/content/cleaned_data/*.png")
noised = glob.glob("/content/noisy_data/*.png")
print(len(cleaned))
print(len(noised))

144
144


In [17]:
def load_data(files, width, height, debug=False):
    X = []
    for file in files:
        im = cv2.imread(file,0)
        im = cv2.resize(im, (width, height),interpolation = cv2.INTER_CUBIC)
        if debug is True:
            cv2_imshow(im)
            debug = False
        im = im.astype(np.float32)
        im = im / 255
        X.append(im)
    X = np.array(X)
    print(X.shape)
    train_x = X.reshape(X.shape[0], width, height, 1)
    return train_x

In [66]:
trainX = load_data(cleaned,440, 440)
trainXNoised = load_data(noised,440, 440)

(144, 440, 440)
(144, 440, 440)


In [24]:
split = int(len(trainX)*0.7)
print(split)

100


In [67]:
train_cleaned = trainX

In [68]:
train_Noise = trainXNoised

In [30]:
train_cleaned = trainX[:split, :, :]
test_cleaned = trainX[split:, :, :]
print(train_cleaned.shape)
print(test_cleaned.shape)

(100, 440, 440, 1)
(44, 440, 440, 1)


In [34]:
train_Noise = trainXNoised[:split, :, :]
test_Noise = trainXNoised[split:, :, :]
print(train_Noise.shape)
print(test_Noise.shape)

(100, 440, 440, 1)
(44, 440, 440, 1)


In [69]:
class DenoisingAutoencoder(Model):
  def __init__(self):
    super(DenoisingAutoencoder, self).__init__()
    self.encoder = tf.keras.Sequential([
        layers.Input(shape=(440, 440, 1)), 
        layers.Conv2D(48, (3, 3), activation='relu', padding='same'),
        layers.Conv2D(72, (3, 3), activation='relu', padding='same'),
        layers.Conv2D(144, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2), padding='same'),
        layers.Dropout(0.5),
    ])

    self.decoder = tf.keras.Sequential([
        layers.Conv2D(144, (3, 3), activation='relu', padding='same'),
        layers.Conv2D(72, (3, 3), activation='relu', padding='same'),
        layers.Conv2D(48, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.UpSampling2D((2, 2)),
        layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same')
    ])

  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded


In [70]:
autoencoder = DenoisingAutoencoder()
autoencoder.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error'])

In [75]:
es = callbacks.EarlyStopping(
monitor='loss', patience=50, verbose=1, restore_best_weights=True
)
rlp = callbacks.ReduceLROnPlateau(
     monitor='loss', factor=0.5, patience=5, min_lr=1e-6, mode='min', verbose=1
)
history =  autoencoder.fit(
train_Noise, train_cleaned, 
     validation_split=0.2, 
shuffle=True,
callbacks=[es, rlp], epochs=800, batch_size=12)

Epoch 1/800
10/10 [==============================] - 10s 1s/step - loss: 0.0366 - mean_absolute_error: 0.1098 - val_loss: 0.0341 - val_mean_absolute_error: 0.1049
Epoch 2/800
10/10 [==============================] - 10s 1s/step - loss: 0.0366 - mean_absolute_error: 0.1098 - val_loss: 0.0341 - val_mean_absolute_error: 0.1049
Epoch 3/800
10/10 [==============================] - 11s 1s/step - loss: 0.0366 - mean_absolute_error: 0.1098 - val_loss: 0.0341 - val_mean_absolute_error: 0.1049
Epoch 4/800
10/10 [==============================] - 10s 1s/step - loss: 0.0366 - mean_absolute_error: 0.1098 - val_loss: 0.0341 - val_mean_absolute_error: 0.1049
Epoch 5/800
10/10 [==============================] - 10s 1s/step - loss: 0.0366 - mean_absolute_error: 0.1098 - val_loss: 0.0341 - val_mean_absolute_error: 0.1049
Epoch 6/800
10/10 [==============================] - 10s 1s/step - loss: 0.0366 - mean_absolute_error: 0.1098 - val_loss: 0.0341 - val_mean_absolute_error: 0.1049
Epoch 7/800
10/10 [===

KeyboardInterrupt: ignored

In [76]:
preds = autoencoder.predict(test_Noise)

In [46]:
print("OLA")

OLA


In [51]:
 decoded_img = np.squeeze(autoencoder.decoder(autoencoder.encoder(np.expand_dims(test_Noise[0], axis=0)).numpy()))

In [85]:
c = 0
for i in preds:
    c+=1
    cv2.imwrite("/content/decoded/"+str(c)+".png", (i*255))

In [86]:
!cp -r /content/decoded/ /content/drive/MyDrive/